# Slithering Snake Simulation

In [ ]:
# Import Modules
import numpy as np
import numpy.linalg as la

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

from tqdm.notebook import tqdm

from cosserat import Cosserat
from bspline import test_bspline, snake_bspline


In [ ]:
%load_ext autoreload
%autoreload 2

### Muscular Activity

In [ ]:
import numpy as np
# my_data is a (x, 3) time series data
# The first columnd my_data[:,0] contains the time
# The second columnd my_data[:,1] contains the forward snake velocity
# The third columnd my_data[:,2] contains the lateral snake velocity
my_data = np.loadtxt('optimized_snake.dat')
plt.plot(my_data[:,1])

In [ ]:
# test_bspline needs (BETAS, total_number_of_elements i.e. n, total_length_of_rod i.e. L)
total_length = 3
n_nodes = 100
betas = [5.48010964, 2.88997841, 2.45656694, -0.13776412]
plt.plot(np.linspace(0,total_length,n_nodes), test_bspline(betas, n_nodes, total_length))
plt.plot(np.linspace(0,total_length,len(betas)+2)[1:-1], betas, 'o')

In [ ]:
def torque(s_elements, beta_m, Q, t, lambda_m, T_m):
    # A_m here denotes the magnitude of the torque acting on each element at time 't'
    A_m = beta_m * np.sin((2*np.pi/T_m) * t - (2*np.pi/lambda_m) * s_elements) # [n]

    # extract the y - direction component of the elements
    #d_2 = np.zeros((n,3,))
    #d_2[:,1] = 1.0
    #d_2 = Q [:,1] # [n,3]
    
    # torque vector at each element: --> torque_elem: ((n, 3, ))
    #torque_elem = A_m[:,None] * (Q @ d_2[:,:, None]).reshape(n,3,)
    torque_elem = np.zeros((n,3,))
    torque_elem [:,1] = A_m
    
    # torque vector at each node: --> torque_nodes ((n+1, 3, ))
#     torque_nodes = np.zeros((n+1, 3,))
#     torque_nodes[:-1] += (torque_elem)/2.
#     torque_nodes[1:] += (torque_elem)/2.
    
    return torque_elem

### Contact Force (Wall Force)

In [ ]:
# Function needs n = num_of_elements, f_element, current_r, and current_v
# f_element will be constant throughout out simulations as we just have gravity so we can calc that force beforehand
# this will save time since we won't have to calc that force every time the wall_force function is run
def wall_force (n, f_gravity, r, v):
    # Extract the y-components of gravity for each element
    f_element = f_gravity [:, 1]
    # wall stiffness and dissipation constants
    k_w = 1
    gamma_w = 1e-6
    # Normal vector by the wall
    wall_normal = np.zeros((n, 3, )) # (n,3, )
    wall_normal[:] = [0., 1., 0.]

    # Penetration distance: epsilon
    y_coord = r[:, 1]  # A vector of y - coordinates of each node: dimensions: (n+1, )
    epsilon = (y_coord [:-1] + y_coord [1:]) / 2. # Penetration depth of each element's centre: (n,)

    #Velocity of the centre of each element (in y-direction only)
    velo_element = (v [1:,1] + v [:-1,1])/2. # (n, )

    # wall_force on each element: ((n, 3, ))
    f_wall_elem = np.zeros((n, 3, ))
    # updating just the y-component of wall force as no force is exerted along x and z axis
    f_wall_elem [:, 1] = np.heaviside(- epsilon, 1.) * (- f_element - k_w * epsilon - gamma_w * velo_element)
    
    # wall_force on each node: ((n+1, 3, ))
    f_wall_nodes = np.zeros((n+1, 3,))
    f_wall_nodes[:-1] += (f_wall_elem)/2.
    f_wall_nodes[1:] += (f_wall_elem)/2.

    return f_wall_nodes, f_wall_elem

### Anisotropic Friction

In [ ]:
# Longitudinal Friction Calculation
# calcs friction for each node
def friction_long (v, t, f_element, f_wall_element, n, mu_sf, mu_kf, mu_sb, mu_kb):
    wall_normal = np.zeros((n, 3, ))
    wall_normal[:,1] = 1.0
    
    u_para = np.cross(t, wall_normal) # (n,3)
    u_x = np.cross(wall_normal, u_para) 
    u_x = u_x / la.norm (u_x, axis = 1, keepdims = True) # (n,3) 
    
    # get velocity from the cosserat class: v will be of shape: (n+1,3)
    # convert v to (n, 3, ) -> Avg velocity of the adjacent nodes is assigned to the centre of the element    
    v_elem = (v [:-1] + v [1:])/2. # (n,3)
    v_epsilon = 1e-6  # (threshold velocity)    
    
    vx_mag = np.sum(v_elem * u_x, axis=1, keepdims=True)
    
    mask = np.abs(vx_mag)[:,0] > v_epsilon # masking static friction (True element applies kinetic friction)
    f_long_elem = u_x.copy()
    _abs_f_wall_element = np.abs(f_wall_element[:,1:2])
    # Kinetic friction
    if np.any(mask):
        direction = -np.sign(vx_mag[mask])
        mu = np.ones_like(direction) * mu_kb
        mu[direction < 0] = mu_kf
        f_long_elem[mask] *= (direction * mu * _abs_f_wall_element[mask])

    # Static friction
    if np.any(~mask):
        f_dot_ux = np.sum(f_element[~mask]*u_x[~mask], axis=1, keepdims=True)
        direction = -np.sign(f_dot_ux)
        mu = np.ones_like(direction) * mu_sb
        mu[direction < 0] = mu_sf
        f_long_elem[~mask] *= (direction * np.fmin(np.abs(f_dot_ux), mu * _abs_f_wall_element[~mask]))
        
    # Till here, we successfully calc f_long_element for each element
    # Now, we need to expand the f_long_element from n dims to n+1 dims i.e. apply to all the nodes
    
    f_long_node = np.zeros((n+1, 3, ))
    f_long_node[:-1] += (f_long_elem)/2.
    f_long_node[1:] += (f_long_elem)/2.
         
    return f_long_node

In [ ]:
# Generate Animation
def make_animation(s_list, r_list, n_step):
    fig = plt.figure() 
    #ax = plt.axes(xlim=(-50, 50), ylim=(-50, 50)) 
    ax1 = fig.add_subplot(111)
    plot, = ax1.plot([],[],'-x')
    xmax = max([np.max(ar) for ar in s_list])
    xmin = min([np.min(ar) for ar in s_list])
    ymax = max([np.max(ar) for ar in r_list])
    ymin = min([np.min(ar) for ar in r_list])
    plt.title('zy plot')
    plt.xlabel('z')
    plt.ylabel('y')
    plt.xlim([xmin,xmax])
    plt.ylim([5*ymin,5*ymax])
    def animate(i):
        _ = plot.set_data(s_list[i], r_list[i])

    anim = animation.FuncAnimation(fig, animate, frames=n_step) 
    return fig, anim

# CMAES Optimization Block:

In [ ]:
from setup_snake import setup, L, n, M, mu_sf, mu_kf, mu_sb, mu_kb, dt, T_m

s_hat = np.linspace(0., L, n+1) # Positions of nodes along centerline when t = 0 (i.e. initial configuration)
s_elements = (s_hat[:-1] + s_hat[1:])/2. # [n] --> Center points of each element along centerline

# Gravitational force on each element (only the y-component)
f_gravity_elements = np.zeros((n, 3, )) # (n,3,)
f_gravity_elements [:,1] = - (M/n) * 9.81

# Gravitational Force Vector on each node (will be used later for calculation of total_force)
f_gravity_nodes = np.zeros((n+1, 3, ))
f_gravity_nodes [:-1] += f_gravity_elements/2.
f_gravity_nodes [1:] += f_gravity_elements/2.

s_list = [] # Position list for future use
y_list = [] # Height list for future use
t_list = [] # time list
couples = []
friction = []
total_force_list = []

def snake_CMA_call(x):
    
    # initialization
    solution = Cosserat(**setup)
    n_step = 10000
    r, v, Q, w = solution.get_state # Initial state
    tangent = solution.get_tangent
    
    s_list.append(r[:,2]) # save k-axis
    y_list.append(r[:,0]) # save i-axis
    
    # Get the parameter x from CMAES
    betas = x[0:4]
    lambda_m = x[4]
    # Constraint: Floor the values of Betas to 50.0 if they exceed 50.0
    betas = np.fmin(betas, 50.0)
    
    beta_m = np.array(test_bspline(betas, n , L)) # [n] --> for each element --> used in torque func
    
    v_frwd = [] # Will append the avg. frwd velocity of snake for time 't': 4 < t <=6. (i.e. 2 activation cycles)
    
    #Initialize internal shear force at each element
    shear_nodes = np.zeros((n+1,3,))
    time = 0
    
    for itr in tqdm(range(n_step)):
        
        shear_nodes[0], shear_nodes[-1]  = 2 * shear_nodes[0], 2 * shear_nodes[-1]  
        # Calc shear at the elements: [n]
        shear_elements = (shear_nodes [:-1] + shear_nodes[1:])/2.
               
        # (i) Wall Force Calculation:
        f_wall_nodes, f_wall_elements = wall_force (n, (f_gravity_elements + shear_elements), r, v)
        
        # (ii) Friction Force Calculation:
        F = f_wall_elements + f_gravity_elements + shear_elements # Total force on each element
        f_friction_nodes = friction_long (v, tangent, F, f_wall_elements, n, mu_sf, mu_kf, mu_sb, mu_kb)
            
        # (iii) Total Force Calculation at each node:
        f_total_nodes = f_wall_nodes + f_gravity_nodes + f_friction_nodes
        f_total_nodes -= 1000 * v
        
        # (iv) Torque Calculation:
        couple = torque(s_elements, beta_m, Q, time, lambda_m, T_m) # [n]

        # Run step
        r, v, Q, w  = solution.step(dt = dt, couple = couple, force = f_total_nodes)
        #r, v, Q, w  = solution.step(couple=couple)
        tangent = solution.get_tangent
        shear_nodes = solution.get_shear
        
        if itr % 300 == 0:
            s_list.append(r[:,2])
            y_list.append(r[:,0])
            t_list.append(time)
            v_frwd.append([np.average(np.linalg.norm(v[:], axis = 1))])
       
        time += dt
    
#     v_frwd = np.array(v_frwd)
#     plt.plot(v_frwd.T[0], label = 'Z-direction')
#     plt.subplots(figsize=(20,10))
#     plt.plot(v_frwd.T[0], label = 'Average Actual velocity')
#     plt.xlabel('Number of Iterations')
#     plt.ylabel('Velocity (m/s)')
#     plt.legend()
#     plt.show()
    
    # Calculate fitness function
    fitness = - np.average(np.linalg.norm(v[:], axis = 1))
    
    return fitness

# One Run of the Snake's Simulation using Optimized Betas and Lambda from CMA

In [ ]:
# Test optimal run
#opt_beta = [17.4, 48.5, 5.4, 14.7]
# run_1_beta = [5.48010964, 2.88997841, 2.45656694, -0.13776412]
# run_1_lambda = 2.21413977

run_2_beta = [-0.25702005, 12.85092678,  3.07562002, -3.86988345]
run_2_lambda = -2.58794542

opt_beta = run_2_beta
opt_lambda = run_2_lambda

opt_x = np.array(opt_beta+[opt_lambda])
snake_CMA_call(opt_x)

# Visualizing snake in 2D animation

In [ ]:
_, anim = make_animation(s_list, y_list, len(s_list))

In [ ]:
HTML(anim.to_jshtml())

# CMA Optimization Campaign

In [ ]:
from cma import CMAES

# CMAES Params: Initialization
initial_centroid = np.random.random(5, ) # 5 dimensions to optimize for
initial_centroid [:4] = initial_centroid [:4] * 10.0
print(initial_centroid)

# Parameters to tweek:
iterations = 30
sigma = 10
pop_size = 25

cma_es = CMAES (initial_centroid, sigma, pop_size, iterations)
best_pop, best_outputs = cma_es.run(snake_CMA_call)
print("Values of Betas and Lambda_m are: ", best_pop)
print("The value of velocity is: ", - best_outputs[-1])

In [ ]:
plt.plot(best_outputs, 'o--')
plt.ylabel("Fitness Function (-ve of avg-total-velocity (m/s))")
plt.xlabel('Iteration Number')
plt.title('CMA Optimization of fitness function')